# Data cleaning 

In [1]:
import pandas as pd

In [2]:
from use_cases.contributions import create_table_contributions

from use_cases.emotions import create_table_emotions
from use_cases.personal_needs import create_table_personal_needs
from use_cases.dialogues import create_table_dialogues
from use_cases.country_needs import create_table_country_needs
from use_cases.persons import create_table_persons
from use_cases.individuals import create_table_individuals
from use_cases.pairs import create_pair_token


pd.options.mode.chained_assignment = None 

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading Data Surveys

In [3]:
chunksize   = 100 # Just for debugging
survey_path = './data/BBDD_Dialogos_2021_01_27.csv' 
survey = pd.read_csv(survey_path, chunksize=chunksize, low_memory=False)
for survey in survey: break
    
ind_online_survey_path = './data/Consulta_Individual_online_v2.xlsx'
ind_online_survey = pd.read_excel(ind_online_survey_path, 'Sheet1')
    
ind_survey_path = './data/Base_final_digitación_Consultas.xlsx'
ind_survey = pd.read_excel(ind_survey_path, 'CONSULTAS')

In [4]:
survey.sample(2)
ind_online_survey.sample(2)

,Submission Date,RUN,Edad,Comuna,1. ¿Cuál es el nivel de educación alcanzado por Usted?,2. ¿Cuál es el nivel de educación que alcanzó el jefe de su hogar? (sólo si se trata de alguien distinto a Usted),1 >> Emociones / Sentimientos / Sensaciones,1 >> Explique lo mencionado,2 >> Emociones / Sentimientos / Sensaciones,2 >> Explique lo mencionado,...,2 >> Rol del actor social (Describa),3 >> Necesidades del país identificadas,3 >> Rol del Estado (Describa),"3 >> Actor social (empresa, organizaciones sociales, medios de comunicación, comunidad, etc)",3 >> Rol del actor social (Describa),5. Pregunta: ¿Cuál es mi contribución personal para construir el Chile que queremos?,Comuna.1,"Comentarios finales: Si tiene alguna observación o sugerencia, en este espacio puede indicarla.",Región,Región.1
44,2020/02/03 17:16:22,e4a58e536e1ef5f75cc4aa12c6b3a5f0,50,NaN,Universitaria incompleta. Técnica completa.,Yo soy el jefe de hogar.,alegria,por el despertar de la sociedad,Preocupacion,"Cuando se agravo la violencia, temor por lo qu...",...,NaN,Delincuencia,nuevas reformas,NaN,NaN,preocuparme por el medio ambiente en contamina...,Valdivia,"Con respecto a la delincuencia, así como cada ...",Región de Los Ríos,NaN
146,2020/01/27 17:20:08,febeaafa2aac92987d2b03199c20b791,52,NaN,Universitaria completa.,Yo soy el jefe de hogar.,Agotada,Por el incremento de trabajo en la Unidad.,Melancólica,Quizás por falta de recreación,...,NaN,NaN,NaN,NaN,NaN,Apoyar aquellas propuestas que beneficien en g...,Rancagua,La sugerencia es seguir manteniendo este tipo ...,Región del Libertador General Bernardo O’Higgins,NaN


## Processing Uses Cases
1. Emotions 
2. Contributions
3. Country needs 
4. Personal/familiar needs

### Persons Table (Dialogue)

In [5]:
persons_table = create_table_persons(survey)

In [6]:
persons_table.sample(3)

,id,diag_id,age,sex,level,comuna_id,age_range
82,481,enc_u_4596766050112223446,33,h,media_completa,1,30-45
22,561,enc_u_4601731044543916003,43,f,universitaria_completa,8302,30-45
98,198,enc_u_4595906623579183150,51,f,basica_completa,8105,45-60


### Dialogues

In [7]:
dialogues_table = create_table_dialogues(survey)

In [8]:
dialogues_table.sample(3)

,id,date,init_time,end_time,location,address,comuna_id,n_members,group_name
51,enc_u_4598337190995449677,22-01-2020,17:00:00,00:00:00,direccion regional injuv coquimbo,av francisco de aguirre #414,4101,7,usuarios de oficina movil de bienes nacionales
50,enc_u_4598338985522268051,06-03-2020,10:30:00,12:00:00,pucon,calle ansorena s/n,9115,5,usuarios de oficina movil de bienes nacionales
80,enc_u_4596799489416460526,10-01-2020,15:00:00,17:45:00,sala de reuniones seremi mop,bulnes 897,9101,5,usuarios de oficina movil de bienes nacionales


### Individuals

In [9]:
%%time
individuals_table = create_table_individuals(ind_online_survey, ind_survey)

CPU times: user 7.49 s, sys: 76.5 ms, total: 7.57 s
Wall time: 7.74 s


In [10]:
individuals_table.sample(2)

,id,date,age,comuna_id,level,online,age_range
10005,5227912,21-01-2020,45,13129,Técnica completa,,45-60
5586,3251581,23-01-2020,71,4301,Básica completa,,>=60


### Emotions

In [14]:
emotions_table = create_table_emotions(survey, ind_survey_path, ind_online_survey)

In [16]:
emotions_table.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online,source_id
8439,19645,,,preocupacion,[preocupacion],,incertidumbre ante acontecimientos,"[incertidumbre, acontecimientos]",0,4162910
857,13666,,243960e69adeee65e31bb13224c1e96a,decepcionada,[decepcionada],,"de la respuestas del gobierno dia a dia, fue ...","[respuestas, gobierno, dia, dia, decepcionante...",1,


### Emotion Pair

In [18]:
emotion_pair = create_pair_token(emotions_table, 'exp_tokens', 'emotion_id')

In [19]:
emotion_pair.sample(3)

,id,emotion_id,word_1,word_2
28834,28835,7718,bonito,cambie
73774,73775,20184,queda,gente
9270,9271,2258,cambios,concretos


### Country Needs

In [20]:
%%time
country_needs = create_table_country_needs(survey, ind_survey, ind_survey_path, ind_online_survey_path)

CPU times: user 1min 13s, sys: 3.46 s, total: 1min 17s
Wall time: 1min 17s


In [21]:
country_needs.sample(3)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority,is_online
13711,13749,,3226816,salud,[salud],,,,,,,1,0
16674,32002,,3222266,justicia real,"[justicia, real]",,,,,,,,0
6103,6805,,3147777,igualdad,[igualdad],,,,mas union entre la clase trabajadora y los sin...,"[mas, union, clase, trabajadora, sindicatos, t...",trabajador y sindicatos,1,0


### Country Need Role Pair

In [22]:
country_need_role_pair = create_pair_token(country_needs, 'role_tokens', 'country_need_id')

In [23]:
country_need_role_pair.sample(2)

,id,country_need_id,word_1,word_2
60047,60048,36453,hospitales,mayor
61661,61662,37689,mas,activos


### Country Need Explanation Pair

In [24]:
country_need_exp_pair = create_pair_token(country_needs, 'exp_tokens', 'country_need_id')

In [25]:
country_need_exp_pair.sample(2)

,id,country_need_id,word_1,word_2
19192,19193,32691,tribunales,privilegian
33696,33697,44320,necesarios,entonces


### Family/Personal Needs

In [28]:
%%time
personal_needs = create_table_personal_needs(survey, ind_survey, ind_online_survey)

CPU times: user 1min, sys: 4.27 s, total: 1min 4s
Wall time: 1min 4s


In [29]:
personal_needs.sample(3)

,id,diag_id,ind_id,name,name_tokens,exp,exp_tokens,macro,priority,is_online
370,692,,2843589,pensiones,[pensiones],dos adultos mayores con pensiones asistenciales,"[dos, adultos, mayores, pensiones, asistenciales]",pensiones,1.0,0
9723,8729,,4737176,tiempo,[tiempo],para estar en familia,[familia],tiempo,1.0,0
12491,24143,,3159838,pensiones,[pensiones],mis abuelos tienen muy poca pension y no le al...,"[abuelos, poca, pension, alcanza, remedios, co...",pensiones,2.0,0


### Family/Personal Need Pair

In [30]:
personal_need_pair = create_pair_token(personal_needs, 'exp_tokens', 'personal_need_id')

In [31]:
personal_need_pair.sample(3)

,id,personal_need_id,word_1,word_2
81096,81097,19095,respaldo,persona
107476,107477,25458,sueldo,decente
92744,92745,22135,hogar,necesitamos


### Contributions

In [32]:
contributions = create_table_contributions(survey, ind_survey, ind_online_survey)

In [33]:
contributions.sample(3)

,id,diag_id,ind_id,text,tokens,macro,is_online
335,14424,,dd7987bb58a487b829f4f5ee2a3db380,poder brindar el mayor esfuerzo para obtener u...,"[poder, brindar, mayor, esfuerzo, obtener, mej...",poder brindar el mayor esfuerzo para obtener u...,1
4449,4949,,3194106,"en mi casa doy trabajo a 15 grupos familiares,...","[casa, doy, trabajo, grupos, familiares, debo,...","en mi casa doy trabajo a 15 grupos familiares,...",0
8867,9367,,4241605,una nueva constitucion hecha por el pueblo,"[nueva, constitucion, hecha, pueblo]",una nueva constitucion hecha por el pueblo,0
